In [2]:
# TODO: Dodać komentarze do kodu i zrobić jego refactor
# W przód
struct Dual{T <:Number} <:Number
    v::T
    dv::T
end

In [3]:
import Base: +, -, *, /
     -(x::Dual) = Dual(-x.v, -x.dv)
     +(x::Dual, y::Dual) = Dual( x.v + y.v, x.dv + y.dv)
     -(x::Dual, y::Dual) = Dual( x.v - y.v, x.dv - y.dv)
     *(x::Dual, y::Dual) = Dual( x.v * y.v, x.dv * y.v + x.v * y.dv)
     /(x::Dual, y::Dual) = Dual( x.v / y.v, (x.dv * y.v - x.v * y.dv)/y.v^2)
import Base: abs, sin, cos, tan, exp, sqrt, isless
    abs(x::Dual) = Dual(abs(x.v),sign(x.v)*x.dv)
    sin(x::Dual) = Dual(sin(x.v), cos(x.v)*x.dv)
    cos(x::Dual) = Dual(cos(x.v),-sin(x.v)*x.dv)
    tan(x::Dual) = Dual(tan(x.v), one(x.v)*x.dv + tan(x.v)^2*x.dv)
    exp(x::Dual) = Dual(exp(x.v), exp(x.v)*x.dv)
    sqrt(x::Dual) = Dual(sqrt(x.v),.5/sqrt(x.v) * x.dv)
    isless(x::Dual, y::Dual) = x.v < y.v;

In [4]:
import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");
value(x::Dual) = x.v;
partials(x::Dual) = x.dv;

In [6]:
import Base: convert, promote_rule
convert(::Type{Dual{T}}, x::Dual) where T =
 Dual(convert(T, x.v), convert(T, x.dv))
convert(::Type{Dual{T}}, x::Number) where T =
 Dual(convert(T, x), zero(T))
promote_rule(::Type{Dual{T}}, ::Type{R}) where {T,R} =
 Dual{promote_type(T,R)}

promote_rule (generic function with 125 methods)

In [8]:
# TODO: Przetestować dla funkcji ReLU, soft-max, podstawowych funkcji matematycznych
f(x) = sin(x*x)
ϵ = Dual(0., 1.)
x = 5.0 + ϵ
y = f(x)

(-0.13235175009777303) + [9.912028118634735ϵ]

In [9]:
sin(25.0), 10cos(25.0)

(-0.13235175009777303, 9.912028118634735)

In [11]:
# TODO: Ogarnąć wyznaczenie macierzy jakobiego
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
            Dual(args[j], one(args[j])) :
            Dual(args[j],zero(args[j])) )
        end
        column = partials.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

In [ ]:
# TODO: potestować metodę (w tym dla funkcji Rosenbrocka), zapisać wyniki

In [13]:
# w tył
# TODO: Zaimplementować Operator, naprawić resztę błędów w rózniczkowaniu w tył
abstract type Node end
abstract type LeafNode <: Node end

mutable struct Variable{T} <: LeafNode
     value::T
     grad::T
end

struct ComputableNode{OT, AT} <: Node
     op::OT
     args::AT
end

mutable struct CachedNode{NT, OUT} <: Node
     node::NT
     out::OUT
end

In [19]:
# TODO: Przetestować dla funkcji ReLU, soft-max, podstawowych funkcji matematycznych
f(x) = sin(x*x)
x = Variable(5.0, 0.0)
y = f(x)

LoadError: MethodError: no method matching forward(::Variable{Float64})
[0mClosest candidates are:
[0m  forward([91m::CachedNode[39m) at In[17]:1
[0m  forward([91m::ComputableNode[39m) at In[17]:2

In [22]:
# TODO: Dodać więcej przeciążeń
function register(op, args...)
     node = ComputableNode(op, args)
     out = forward(node)
     CachedNode(node, out)
end

import Base: +, -, *, /
     +(x::Node, y::Node) = register(+, x, y)
     -(x::Node, y::Node) = register(-, x, y)
     *(x::Node, y::Node) = register(*, x, y)
     /(x::Node, y::Node) = register(/, x, y)
import Base: sin
    sin(x::Node) = register(sin, x)

sin (generic function with 15 methods)

In [23]:
forward(cached::CachedNode) = cached.out = forward(cached.node)
forward(node::ComputableNode) = forward(node.op, map(forward, node.args)...)
forward(op::Operator, args...) = op.f(args...; kwargs...)
forward(var::Variable) = var.value

LoadError: UndefVarError: Operator not defined

In [24]:
function backward(cached::CachedNode, grad)
     grad_inputs = gradient(cached, grad)
     for (each, each_grad) in zip(cached.node.args, grad_inputs)
         backward(each, each_grad)
     end
end

gradient(cached::CachedNode, grad) =
     gradient(cached.node.op, grad, map(value, cached.node.args)...)

gradient(op::Operator, grad, args...) =
     gradient(op.f, grad, args...)

value(cached::CachedNode) = value(cached.out)
value(var::Variable) = var.value

gradient(::typeof(*), grad, x, y) = (grad * y, grad * x)
gradient(::typeof(sin), grad, x) = (grad * cos(x), )

function backward(var::Variable, grad)
     var.grad += grad
end

LoadError: UndefVarError: Operator not defined

In [ ]:
#TODO: macierze Jakobiego dla różniczkowania w tył

In [ ]:
# TODO: potestować metodę (w tym dla funkcji Rosenbrocka), zapisać wyniki

In [ ]:
# TODO: Zrobić krótkie zestawienie metod